# Depedencies

In [48]:
import pandas as pd
import numpy as np
import time
import collections
import plotly.graph_objects as go
import plotly.express as px
import math
import seaborn as sn

# Import the Data from the excel

In [49]:
# Read the printify products excel file into pandas dataframe
orders = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Orders')
orders.head()

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2013244.178,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1358429.695,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,939390.193,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1856107.400,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1843904.400,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56


In [50]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13506 entries, 0 to 13505
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   MERCHANT_ID             13506 non-null  int64         
 1   ORDER_ID                13506 non-null  float64       
 2   SHOP_ID                 13506 non-null  int64         
 3   ADDRESS_TO_COUNTRY      13505 non-null  object        
 4   ADDRESS_TO_REGION       12930 non-null  object        
 5   ORDER_DT                13506 non-null  datetime64[ns]
 6   FULFILLED_DT            12637 non-null  datetime64[ns]
 7   REPRINT_FLAG            103 non-null    float64       
 8   SALES_CHANNEL_TYPE_ID   13506 non-null  int64         
 9   TOTAL_COST              13506 non-null  int64         
 10  TOTAL_SHIPPING          13506 non-null  int64         
 11  MERCHANT_REGISTERED_DT  13506 non-null  datetime64[ns]
 12  SUB_IS_ACTIVE_FLAG      13506 non-null  bool  

In [51]:
# Read the printify products excel file into pandas dataframe
new_line_items = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Line items')
new_line_items.head(20)

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
5,1.750016e+06,30,Generic brand,Mug,on-hold,1
6,4.668482e+05,10,Generic brand,Bags,shipment_delivered,1
7,1.569960e+06,45,Gildan,Sweatshirt,fulfilled,1
8,2.004430e+06,29,Gildan,Sweatshirt,shipment_delivered,1
9,2.255664e+05,25,Bella+Canvas,T-Shirt,shipment_delivered,1


In [52]:
new_line_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57583 entries, 0 to 57582
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ORDER_ID           57583 non-null  float64
 1   PRINT_PROVIDER_ID  57583 non-null  int64  
 2   PRODUCT_BRAND      57583 non-null  object 
 3   PRODUCT_TYPE       57583 non-null  object 
 4   ITEM_STATUS        57583 non-null  object 
 5   QUANTITY           57583 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 2.6+ MB


# Clean The data

## line_items cleaning

In [53]:
# Remove duplicates 
line_items = new_line_items.drop_duplicates(subset=['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS'])

# DOn't remove orders with similar characteristics except quantity
#line_items = new_line_items.drop_duplicates(subset=['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS', 'QUANTITY'])
line_items.reset_index(drop=True, inplace=True)
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
...,...,...,...,...,...,...
15616,1.735807e+06,2,Generic brand,Canvas,shipment_delivered,1
15617,1.732949e+06,26,Gildan,T-Shirt,fulfilled,1
15618,1.449381e+06,29,Gildan,Sweatshirt,shipment_out_for_delivery,1
15619,1.735807e+06,48,Generic brand,Mug,shipment_delivered,1


## Orders cleaning

In [54]:
# First rows encounter with duplicated order_id 
duplicate_orders1 = orders[orders['ORDER_ID'].duplicated(keep='last')]
duplicate_orders1

# Second rows encounter with duplicated order_id 
duplicate_orders2 = orders[orders['ORDER_ID'].duplicated(keep='first')]
duplicate_orders2

# Combine the duplicated rows into one dataframe for closer comparison
duplicates = pd.concat([duplicate_orders1, duplicate_orders2]).sort_values(by=['MERCHANT_ID'])
duplicates

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
537,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-24 17:34:42
538,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-28 16:51:06
5436,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,USPS,2020-10-16 19:44:40
5437,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,UPS_MAIL_INNOVATIONS,2020-10-19 19:21:56
13,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,USPS,2020-10-09 21:38:49
14,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,NaN,NaT
7502,7016547,1.555863e+06,1555863,US,TX,2020-10-29 22:10:53,2020-11-02 18:07:59,NaN,1,167,350,2020-02-17 23:36:26,True,business_account_3,USPS,2020-11-05 20:05:27
5285,7016547,1.555863e+06,1555863,US,LA,2020-10-11 01:33:06,2020-10-14 21:42:00,NaN,1,103,350,2020-02-17 23:36:26,True,business_account_3,NaN,NaT
6069,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-09 22:02:28
6070,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-13 20:28:54


In [55]:
# Drop duplicates of merchant id and order id
orders.drop_duplicates(subset=['MERCHANT_ID', 'ORDER_ID'], 
                               inplace=True, keep='last')

In [56]:
# Not complete in removing the errors
orders.drop_duplicates(subset=['MERCHANT_ID', 'ORDER_ID', 
                               'SHOP_ID', 'ADDRESS_TO_COUNTRY', 
                               'ADDRESS_TO_REGION','ORDER_DT',
                               'FULFILLED_DT','REPRINT_FLAG',
                               'SALES_CHANNEL_TYPE_ID',
                               'TOTAL_COST','TOTAL_SHIPPING',
                               'MERCHANT_REGISTERED_DT',
                               'SUB_IS_ACTIVE_FLAG','SUB_PLAN', 
                               'SHIPMENT_CARRIER'], inplace=True,
                                  keep='last')
orders.reset_index(drop=True, inplace=True)

In [57]:
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,7562898,2.225081e+06,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13491,7475289,2.153683e+06,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13492,7144356,1.717608e+06,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13493,7581194,2.243421e+06,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


## Prepare for Joining

In [58]:
# Iteratively, mutate the order id to remove the shop id in the number
# The order id was concatenated with shop id using a dot making seem like
# a floating data type
for i in range(len(orders['ORDER_ID'])):
    
    # We convert the order id to string, split into 2 from where the dot is
    # The right from the dot section is shop id and left is order id
    order = str(orders.loc[i]['ORDER_ID']).split(".")
    
    # At same location, put the order id only, we already have the shop id on the dataset
    orders.at[i, 'ORDER_ID'] = int(order[-1])
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,695.0,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,193.0,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,4.0,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,4.0,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,7562898,12.0,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13491,7475289,2224.0,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13492,7144356,5.0,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13493,7581194,66.0,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


In [59]:
pd.options.mode.chained_assignment = None  # default='warn'
for i in range(len(line_items['ORDER_ID'])):
    
    # Split at the dot and get shop id (right) and order id (left)
    order = str(line_items.loc[i]['ORDER_ID']).split(".")
    
    # At same location insert the order id
    line_items.at[i, 'ORDER_ID'] = int(order[-1])
    
    # Add another column called shop id and add the right part
    # using this (shop_id) is the only way to join the data right
    line_items.at[i,'SHOP_ID'] = int(order[0])
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY,SHOP_ID
0,15208.0,30,Generic brand,Mug,on-hold,3,1750015.0
1,12357.0,30,Generic brand,Mug,on-hold,2,1750015.0
2,10081.0,16,Gildan,Kids clothes,shipment_delivered,1,1225155.0
3,707.0,39,Bella+Canvas,T-Shirt,shipment_delivered,1,1828890.0
4,55.0,10,Generic brand,Accessories,shipment_delivered,1,1847847.0
...,...,...,...,...,...,...,...
15616,1367.0,2,Generic brand,Canvas,shipment_delivered,1,1735807.0
15617,7852.0,26,Gildan,T-Shirt,fulfilled,1,1732948.0
15618,11854.0,29,Gildan,Sweatshirt,shipment_out_for_delivery,1,1449381.0
15619,2759.0,48,Generic brand,Mug,shipment_delivered,1,1735807.0


# Join the Data

In [60]:
# Join the two dataframes to form one where each merchant is 
# Matched with all their orders and shops
printify_df = pd.merge(orders, line_items, how="inner", on=['ORDER_ID','SHOP_ID'])

In [61]:
printify_df

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,...,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,1,Generic brand,Mug,shipment_delivered,1
1,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,16,Generic brand,Mug,shipment_delivered,1
2,5888534,695.0,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,...,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56,39,Gildan,Sweatshirt,shipment_delivered,1
3,6484224,193.0,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,...,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54,25,Delta,T-Shirt,shipment_delivered,1
4,7251033,4.0,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,...,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37,16,Generic brand,Mug,shipment_delivered,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15616,7562898,12.0,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,...,2020-10-29 16:51:47,False,NaN,NaN,NaT,10,Generic brand,Accessories,canceled,10
15617,7475289,2224.0,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,...,2020-09-30 23:11:41,True,business_account_3,NaN,NaT,16,Sport-Tek,Hats,on-hold,1
15618,7144356,5.0,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,...,2020-05-20 01:38:41,False,NaN,NaN,NaT,3,Gildan,Hoodie,on-hold,1
15619,7581194,66.0,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,...,2020-11-05 20:02:09,False,NaN,NaN,NaT,3,Gildan,Hoodie,canceled,1


In [62]:
printify_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15621 entries, 0 to 15620
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   MERCHANT_ID             15621 non-null  int64         
 1   ORDER_ID                15621 non-null  float64       
 2   SHOP_ID                 15621 non-null  int64         
 3   ADDRESS_TO_COUNTRY      15620 non-null  object        
 4   ADDRESS_TO_REGION       15009 non-null  object        
 5   ORDER_DT                15621 non-null  datetime64[ns]
 6   FULFILLED_DT            14651 non-null  datetime64[ns]
 7   REPRINT_FLAG            108 non-null    float64       
 8   SALES_CHANNEL_TYPE_ID   15621 non-null  int64         
 9   TOTAL_COST              15621 non-null  int64         
 10  TOTAL_SHIPPING          15621 non-null  int64         
 11  MERCHANT_REGISTERED_DT  15621 non-null  datetime64[ns]
 12  SUB_IS_ACTIVE_FLAG      15621 non-null  bool  

# Printers Performance Metrics

In [63]:
# Date offsetting
max_day = printify_df['SHIPMENT_DELIVERD_DT'].max() + pd.DateOffset(hours=100)
print(printify_df['SHIPMENT_DELIVERD_DT'].max())
print(max_day)

2021-01-26 20:13:23
2021-01-31 00:13:23


In [64]:
# In the case a shipment is not delivered, deliverly is assumed to take 
# place in the next 4 days hence the max day value for null deliver date
max_day = printify_df['FULFILLED_DT'].max() + pd.DateOffset(hours=72)

# Capture the necessary columns
printify_df_temp1 = printify_df[['PRINT_PROVIDER_ID', 'ORDER_DT','FULFILLED_DT']]

# For each order, calculate the time delay by the printer
printify_df_temp1['Time_delay_hours'] = (printify_df['FULFILLED_DT'].fillna(max_day) - printify_df['ORDER_DT']).astype('timedelta64[h]')

# For each printer, get the mean delay in an order
printer_delay = printify_df_temp1[['PRINT_PROVIDER_ID', 'Time_delay_hours']].groupby(['PRINT_PROVIDER_ID']).mean()
printer_delay

,Time_delay_hours
PRINT_PROVIDER_ID,
1,175.976900
2,199.278557
3,215.475799
6,293.689394
7,52.769231
8,981.000000
9,204.800000
10,181.475077
14,819.203390


In [65]:
# Get the sum of all products sold, it should be more or equal for each printers total orders
total_orders_printed = printify_df[['PRINT_PROVIDER_ID', 'QUANTITY']].groupby(['PRINT_PROVIDER_ID']).sum()

# Rename columns
total_orders_printed.rename(columns={'QUANTITY':'Total_printed_orders'}, inplace=True)
total_orders_printed

,Total_printed_orders
PRINT_PROVIDER_ID,
1,1629
2,528
3,2240
6,132
7,13
8,1
9,73
10,1854
14,59


In [66]:
# First capture the needed columns into a new df
printify_df_temp = printify_df[['PRINT_PROVIDER_ID', 'REPRINT_FLAG']]

# count the times a reprint flag is present for all the orders done by each printer
reprint = printify_df_temp[(printify_df['REPRINT_FLAG'] == 1)].groupby(['PRINT_PROVIDER_ID']).count()

# Rename columns
reprint.rename(columns={'REPRINT_FLAG':'Re_printed'}, inplace=True)
reprint

,Re_printed
PRINT_PROVIDER_ID,
1,11
2,10
3,11
7,1
10,12
14,1
16,17
25,5
26,1


In [67]:
printify_df['PRODUCT_TYPE'].unique()

array(['Mug', 'Sweatshirt', 'T-Shirt', 'Poster', 'Home Decor',
       'Phone Case', 'Long-sleeve', 'Paper products', 'All Over Prints',
       'Accessories', 'Tank Top', 'Hoodie', 'Kids clothes', 'Shoes',
       'Hats', 'Bags', 'Puzzle', 'V-neck', 'Canvas', 'Trousers',
       'Laptop Sleeve', 'Calendar'], dtype=object)

In [68]:
# Get the mode of brand column for data grouped for each print provider
printer_poplular_brand = printify_df[['PRINT_PROVIDER_ID', 'PRODUCT_BRAND']].groupby(['PRINT_PROVIDER_ID'], sort=True).agg(pd.Series.mode)

# Rename columns
printer_poplular_brand.rename(columns={'PRODUCT_BRAND':'Popular_brand'}, inplace=True)
printer_poplular_brand

,Popular_brand
PRINT_PROVIDER_ID,
1,Generic brand
2,Generic brand
3,Gildan
6,Gildan
7,Generic brand
8,Fruit Of The Loom
9,Generic brand
10,Generic brand
14,Generic brand


In [69]:
# Get the mode of product type column for data grouped for each print provider
printer_poplular_product = printify_df[['PRINT_PROVIDER_ID', 'PRODUCT_TYPE']].groupby(['PRINT_PROVIDER_ID'], sort=True).agg(pd.Series.mode)

# Rename columns
printer_poplular_product.rename(columns={'PRODUCT_TYPE':'Popular_brand'}, inplace=True)
printer_poplular_product

,Popular_brand
PRINT_PROVIDER_ID,
1,Mug
2,Poster
3,T-Shirt
6,T-Shirt
7,Poster
8,Kids clothes
9,Mug
10,Accessories
14,All Over Prints


In [70]:
printify_df.columns

Index(['MERCHANT_ID', 'ORDER_ID', 'SHOP_ID', 'ADDRESS_TO_COUNTRY',
       'ADDRESS_TO_REGION', 'ORDER_DT', 'FULFILLED_DT', 'REPRINT_FLAG',
       'SALES_CHANNEL_TYPE_ID', 'TOTAL_COST', 'TOTAL_SHIPPING',
       'MERCHANT_REGISTERED_DT', 'SUB_IS_ACTIVE_FLAG', 'SUB_PLAN',
       'SHIPMENT_CARRIER', 'SHIPMENT_DELIVERD_DT', 'PRINT_PROVIDER_ID',
       'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS', 'QUANTITY'],
      dtype='object')

In [71]:
# Take the sum of total cost for all orders done by each print provider 
printer_revenue = printify_df[['PRINT_PROVIDER_ID', 'TOTAL_COST']].groupby(['PRINT_PROVIDER_ID']).sum()

# rename columns
printer_revenue.rename(columns={'TOTAL_COST':'Printer_revenue'}, inplace=True)
printer_revenue

,Printer_revenue
PRINT_PROVIDER_ID,
1,2862852
2,693485
3,3496045
6,309024
7,14508
8,0
9,48713
10,4161449
14,558541


# Printer Performance Metric Combining

In [72]:
# Merge the individual printer performance into one df 
# Merge was preffered over concat due to precision and ability
# to specify the collumn to merge on like in SQL
# Thus, not very clean code here, but very optimal as others would
# need to sort first

# merging delay time with total printed orders
printer_perf1 = pd.merge(printer_delay, total_orders_printed, on=['PRINT_PROVIDER_ID'])

# merge previous with reprint data for each printer
printer_perf2 = pd.merge(printer_perf1, reprint, on=['PRINT_PROVIDER_ID'])

# merge printer revenue on previous
printer_perf3 = pd.merge(printer_perf2, printer_revenue, on=['PRINT_PROVIDER_ID'])

# merge on brands and products that each printer does more
printer_perf4 = pd.merge(printer_perf3, printer_poplular_brand, on=['PRINT_PROVIDER_ID'])
printer_perf = pd.merge(printer_perf4, printer_poplular_product, on=['PRINT_PROVIDER_ID'])
printer_perf

,Time_delay_hours,Total_printed_orders,Re_printed,Printer_revenue,Popular_brand_x,Popular_brand_y
PRINT_PROVIDER_ID,,,,,,
1,175.976900,1629,11,2862852,Generic brand,Mug
2,199.278557,528,10,693485,Generic brand,Poster
3,215.475799,2240,11,3496045,Gildan,T-Shirt
7,52.769231,13,1,14508,Generic brand,Poster
10,181.475077,1854,12,4161449,Generic brand,Accessories
14,819.203390,59,1,558541,Generic brand,All Over Prints
16,248.983713,1339,17,2924439,Generic brand,Mug
25,182.034286,357,5,808216,Bella+Canvas,T-Shirt
26,307.625731,272,1,422939,Gildan,T-Shirt


# Print Providers Success criteria

Success_factor = {(log10(Revenue)* (Total_printed - Re_printed * 10)) / (Time_delay * 1.5)}

In [73]:
printer_perf['Printer_sucesss_factor'] = (
(np.log10(printer_perf['Printer_revenue']) * 
(printer_perf['Total_printed_orders'] - printer_perf['Re_printed'] * 10)) /
printer_perf['Time_delay_hours'] * 1.5)
printer_perf.sort_values(['Printer_sucesss_factor'], ascending=False, inplace=True)
printer_perf

,Time_delay_hours,Total_printed_orders,Re_printed,Printer_revenue,Popular_brand_x,Popular_brand_y,Printer_sucesss_factor
PRINT_PROVIDER_ID,,,,,,,
29,147.984587,3925,19,5947910,Gildan,T-Shirt,256.468442
3,215.475799,2240,11,3496045,Gildan,T-Shirt,97.025878
10,181.475077,1854,12,4161449,Generic brand,Accessories,94.870631
1,175.976900,1629,11,2862852,Generic brand,Mug,83.600838
16,248.983713,1339,17,2924439,Generic brand,Mug,45.537941
39,278.034483,1157,3,2210919,Gildan,Sweatshirt,38.576152
2,199.278557,528,10,693485,Generic brand,Poster,18.817608
41,249.261538,530,3,741482,Gildan,Home Decor,17.662474
45,261.330317,452,2,1508839,Gildan,Sweatshirt,15.320689


## How attributes correlate with sucess factor

In [74]:
# Total Revenue
printer_perf['Printer_sucesss_factor'].corr(printer_perf['Printer_revenue'])

0.9323231111336184

In [75]:
fig = px.scatter(printer_perf, x="Total_printed_orders", y="Printer_sucesss_factor", 
                 trendline="ols", size="Printer_revenue",
                labels={'x':'Total Revenue of Printed merchadise', 'y':'Sucess Factor'}, 
                 title='Sucess factor vs Total printed Merchadise, size is orders revenue')
fig.show()

In [76]:
# Total printed
printer_perf['Printer_sucesss_factor'].corr(printer_perf['Total_printed_orders'])

0.9773043623847617

In [77]:
# Time delays
printer_perf['Printer_sucesss_factor'].corr(printer_perf['Time_delay_hours'])

-0.33151743753275953

In [78]:
# using plotly express to plot an interactive scatter plot
fig = px.scatter(printer_perf, x="Time_delay_hours", y="Printer_sucesss_factor", 
                 trendline="ols", size="Total_printed_orders",
                labels={'x':'Average time delay', 'y':'Sucess Factor'}, 
                 title='Sucess factor vs average delay time, size is Printer total orders')
fig.show()

In [79]:
# Reprinted orders
printer_perf['Printer_sucesss_factor'].corr(printer_perf['Re_printed'])

0.8168782609844771

In [80]:
# using plotly express to plot an interactive scatter plot
fig = px.scatter(printer_perf, x="Re_printed", y="Printer_sucesss_factor", 
                 trendline="ols", size="Total_printed_orders",
                labels={'x':'Average time delay', 'y':'Sucess Factor'}, 
                 title='Sucess factor vs re-printed orders, size is Printer total orders')
fig.show()

## Why reprinted has high correlation to success
- Beacause they are directly linked / related to the total number printed orders for a given printer. Despite multiplying its effect by 10, its effect is still high.

# Selecting top and Bottom Two

# Best Performing

In [81]:
# Best two
printer_perf[:2]

,Time_delay_hours,Total_printed_orders,Re_printed,Printer_revenue,Popular_brand_x,Popular_brand_y,Printer_sucesss_factor
PRINT_PROVIDER_ID,,,,,,,
29,147.984587,3925,19,5947910,Gildan,T-Shirt,256.468442
3,215.475799,2240,11,3496045,Gildan,T-Shirt,97.025878


## Apply discount to Printer: 29 and 3

# Dismal performance

In [82]:
# Last two
printer_perf[-2:]

,Time_delay_hours,Total_printed_orders,Re_printed,Printer_revenue,Popular_brand_x,Popular_brand_y,Printer_sucesss_factor
PRINT_PROVIDER_ID,,,,,,,
14,819.203390,59,1,558541,Generic brand,All Over Prints,0.515633
7,52.769231,13,1,14508,Generic brand,Poster,0.354889


## End contract with: 14 and 7